In [1]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00


In [3]:
!pip install --upgrade llama-index



In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [5]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.1 MB/s eta 0:00:00


In [6]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

#OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieving documents from the index
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combining the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
    input_variables=["transcript", "knowledge_base"],
    template=(
        "The following is a clinical conversation between a doctor and a patient. "
        "Based on this conversation transcript and using relevant information from the provided knowledgebase, "
        "generate a detailed History of Present Illness (HPI) for the patient. The HPI should include the following sections:\n\n"
        "1. Chief Complaint: Brief description of only the primary symptom.Dont mention the age of the patient.\n"
        "2. Previous Occurrences: Any similar past episodes.\n"
        "3. Family History: Relevant family medical history.\n"
        "4. Symptoms: only current symptoms.\n\n"
        "Conversation Transcript:\n{transcript}\n\n"
        "Relevant Knowledge Base:\n{knowledge_base}\n\n"
        "Please ensure the HPI is comprehensive, clear, and formatted properly."
    )
)




        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)


transcript_path = '/content/June12-01_AbdominalPain.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)

<ipython-input-6-cffcbb5d081b>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)


KeyError: 'Conversation'

In [8]:
import json

# Load the JSON file
file_path = '/content/June12-01_AbdominalPain.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# Print the structure of the JSON file
print(json.dumps(data, indent=2))


{
  "jobName": "June12-01_AbdominalPain",
  "accountId": "952840929155",
  "status": "COMPLETED",
  "results": {
    "transcripts": [
      {
        "transcript": "Hi, I'm doctor. This is my guy, Jake. Can you tell me what brought you to see us today? Um, yeah, I've just been having some sharp like abdominal pain. It's right underneath my right ribs. Ok. Um, and I am nine weeks pregnant currently, but so I've been having like the usual morning sickness but just that and then yesterday I had some lower back pain, but that's gone now. So it's mainly just the abdominal pain. Ok. Have you had any bleeding or discharge any concern for stds? No. Have you had any vomiting Hi, I'm doctor. We, this is my scribe. Jake. Can you tell me what brought you to see us today? Um, yeah, I've just been having some sharp like abdominal pain. It's right underneath my right ribs. Ok. Um, and I am nine weeks pregnant currently, but so I've been having like the usual morning sickness but just that and then ye

In [15]:
!pip install llama-index-llms-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.5
    Uninstalling langchain-core-0.2.5:
      Successfully uninstalled langchain-core-0.2.5
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.1
    Uninstalling langchain-text-splitters-0.2.1:
      Successfully uninstalled langchain-text-splitters-0.2.1
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.4
    Uninstalling langchain-community-0.2.4:
      Successfully uninstalled langchain-community-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.3
    Uninstalling langchain-0.2.3:
      Successfully uninstalled

In [ ]:
#HPI

In [19]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieving documents from the index
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combining the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['alternatives'][0]['content'] for item in transcript_data['results']['items'] if 'alternatives' in item and item['alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "The following is a clinical conversation between a doctor and a patient. "
                "Based on this conversation transcript and using relevant information from the provided knowledgebase, "
                "generate a detailed History of Present Illness (HPI) for the patient. The HPI should include the following sections:\n\n"
                "1. Chief Complaint: Brief description of only the primary symptom. Don't mention the age of the patient.\n"
                "2. Previous Occurrences: Any similar past episodes.\n"
                "3. Family History: Relevant family medical history.\n"
                "4. Symptoms: only current symptoms.\n\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Relevant Knowledge Base:\n{knowledge_base}\n\n"
                "Please ensure the HPI is comprehensive, clear, and formatted properly."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)

transcript_path = '/content/June12-01_AbdominalPain.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-19-9d7c64328e46>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




History of Present Illness (HPI):

Chief Complaint: Sharp abdominal pain under right ribs and morning sickness at 9 weeks of pregnancy.

Previous Occurrences: Patient has had four previous pregnancies, with one previous miscarriage.

Family History: Patient has one second-degree relative with a history of depression and a "nervous breakdown." There is also a history of hypertension in the family.

Symptoms: Patient is currently experiencing sharp abdominal pain under right ribs and morning sickness, as well as lower back pain that has since resolved. She denies any bleeding, discharge, or concern for STDs. She also reports no recent vomiting other than normal morning sickness associated with pregnancy.

The patient denies any other symptoms such as burning during urination, fever, cough, congestion, runny nose, chest pain, vision changes, or headaches. She also denies being around anyone who is sick. She has no known medical problems, other than being pregnant.

The onset of symptoms

In [ ]:
#ROS

In [21]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# RoSsummary class
class RoSsummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['alternatives'][0]['content'] for item in transcript_data['results']['items'] if 'alternatives' in item and item['alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "Based on the provided clinical conversation and the Conversation Transcript, generate a detailed Review of Systems (RoS) for the patient based on the Conversation Transcript. "
                "The RoS should cover the following body systems and include specific details if mentioned, otherwise state 'denies any issues':\n\n"
                "1. General: Chief complaint\n"
                "2. Skin: Any skin issues or rashes.\n"
                "3. Head: Any headaches or head injuries.\n"
                "4. Eyes: Vision problems or eye pain.\n"
                "5. Ears, Nose, Throat: Hearing issues, nasal congestion, sore throat, etc.\n"
                "6. Cardiovascular: Heart problems, chest pain, etc.\n"
                "7. Respiratory: Breathing issues, cough, etc.\n"
                "8. Gastrointestinal: Abdominal pain, nausea, etc.\n"
                "9. Genitourinary: Urination problems, etc.\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Relevant Knowledge Base:\n{knowledge_base}\n\n"
                "Please ensure the RoS is comprehensive, clear, and formatted properly."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = RoSsummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/June12-01_AbdominalPain.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-21-0633582249e4>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)



Review of Systems (RoS) for Patient:

1. General:
- Chief complaint: Sharp abdominal pain under right ribs
- Pregnancy at 9 weeks
- Morning sickness
- Lower back pain
- No bleeding or discharge
- No concerns for STDs
- No vomiting
- Not taking any medication
- No prior OB/GYN visit
- 5th pregnancy, 3 living children, 1 previous miscarriage

2. Skin:
- No skin issues or rashes mentioned

3. Head:
- No headaches mentioned
- No head injuries mentioned

4. Eyes:
- No vision problems or eye pain mentioned

5. Ears, Nose, Throat:
- No hearing issues mentioned
- No nasal congestion mentioned
- No sore throat mentioned

6. Cardiovascular:
- No heart problems mentioned
- No chest pain mentioned

7. Respiratory:
- No breathing issues mentioned
- No cough mentioned

8. Gastrointestinal:
- Abdominal pain mentioned
- Nausea mentioned

9. Genitourinary:
- Urination problems mentioned

10. Musculoskeletal:
- No pain or discomfort in the back mentioned
- No pain or weakness in the legs mentioned
- No

In [ ]:
#PE

In [27]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# PhysicalExamExtractor class
class PhysicalExamExtractor(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top results, limiting the total length
        combined_text = ""
        for result in search_results:
            combined_text += result.text.replace("Patient", "Other Patient") + "\n"
            if len(combined_text.split()) > 3000:  # Adjust this limit as needed
                break

        return combined_text

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['alternatives'][0]['content'] for item in transcript_data['results']['items'] if 'alternatives' in item and item['alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "The following is a clinical conversation between a doctor and a patient. "
                "Based on this conversation transcript and using relevant information from the provided knowledgebase if any, "
                "generate a detailed physical exam (PE) summary for the patient. The PE should include the findings mentioned by the doctor during the examination. "
                "Please ensure the PE is comprehensive, clear, and formatted properly.\n\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Knowledge Base,if only  any reference is present:\n{knowledge_base}\n\n"
                "Please list only the test names mentioned by the doctor in the Conversation Transcript , excluding any medications or treatments."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = PhysicalExamExtractor(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/June12-01_AbdominalPain.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-27-bc78b7113a98>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)



1. Ultrasound
2. Pelvic exam
3. Wet prep
4. STD screen
5. CBCL FT S
6. Beta quant level
7. Type in Rh
8. IV fluids
9. EMG
10. Nerve conduction studies


In [31]:
#Assessment Plan

In [32]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# AssessmentPlanExtractor class
class AssessmentPlanExtractor(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top results, limiting the total length
        combined_text = ""
        for result in search_results:
            combined_text += result.text.replace("Patient", "Other Patient") + "\n"
            if len(combined_text.split()) > 3000:  # Adjust this limit as needed
                break

        return combined_text

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['alternatives'][0]['content'] for item in transcript_data['results']['items'] if 'alternatives' in item and item['alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "Based on the provided clinical conversation between the doctor and the patient, generate a detailed Assessment Plan. "
                "The Assessment Plan should include only the doctor's assessment and plan, clearly separated into two sections: Assessment and Plan. "
                "Each section should be detailed and follow the structure provided in the conversation.\n\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Assessment:\n"
                "Provide a detailed assessment of the patient's condition based on the Conversation Transcript.\n\n"
                "Plan:\n"
                "1. List only the results of diagnostic tests given to the patient mentioned in Transcript Conversation.\n"
                "2. Include any relevant diagnostic results, such as EKG findings, if mentioned in the Transcript Conversation.\n"
                "3. Mention the diagnosis and follow-up instructions provided by the doctor in Transcript Conversation.\n"
                "Ensure not to include any medications or treatments that are not mentioned explicitly in the transcript.\n\n"
                "Please ensure the assessment plan is comprehensive, clear, and formatted properly."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = AssessmentPlanExtractor(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/June12-01_AbdominalPain.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-32-85de4a4e29f1>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




Assessment:

Based on the patient's reported symptoms and medical history, it appears that she is experiencing abdominal pain in the right upper quadrant. She is currently nine weeks pregnant and has been experiencing morning sickness, lower back pain, and abdominal pain. She has also had one previous miscarriage in her four previous pregnancies. During the physical examination, the patient exhibited right upper quadrant tenderness and a positive Murphy sign. She also reported pain upon deep breathing and palpation in the same area. No other concerning symptoms were reported or observed during the examination.

Plan:

1. Diagnostic tests:
- CBC with differential: to check for any signs of infection or abnormal blood cell counts.
- Beta quant level: to confirm pregnancy and check for any abnormalities in hormone levels.
- Type and Rh: to determine the patient's blood type and Rh factor.
- Pelvic exam: to check for any abnormalities in the reproductive organs.
- Wet prep: to check for 